# Install and import dependencies

In [1]:
!pip install mediapipe opencv-python pandas scikit-learn

In [2]:
import cv2
import mediapipe as mp
import numpy as np

# Set up media pipe
mp_drawing = mp.solutions.drawing_utils
# Import pose estimation model
mp_pose = mp.solutions.pose

In [4]:
# VIDEO FEED

# Open the video file
#video_path = r'videos\1_backhand.mp4'
#cap = cv2.VideoCapture(video_path)
cap = cv2.VideoCapture(0)

while cap.isOpened():
    # Read a frame from the video
    # ret -> return variable
    # frame -> image from cap
    ret, frame = cap.read()
    
    # Display the frame with results
    cv2.imshow('UltiThrow', frame)
    
    # Break the loop if 'q' is pressed
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Release the video capture object and close the window
cap.release()
cv2.destroyAllWindows()

# Make detections

mp_pose.Pose()
min_detection_confidence
min_tracking_confidence

pose

In [6]:
# Open the video file
video_path = r'videos\1_backhand_medium_front.mp4'
cap = cv2.VideoCapture(0)

# Get the original width and height of the video
original_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
original_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Set up mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            print("End of video.")
            break  # Exit the loop when the video is finished.
        

        # Detect stuff and render
        
        # Recolor image
        # default of opencv is rgb, that's why we recolor
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # set the image to not be modified and save memory
        image.flags.writeable = False
        
        # Make detection
        results = pose.process(image) # get detection of pose
        
        # Recolor back to BGR for opencv
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Render detections
        # draw_landamarks -> utils to draw in image
        # results.pose_landmarks -> coordenates
        # mp_pose.POSE_CONNECTIONS -> how body points are conencted inside model
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Update desired height and width before resizing
        desired_height, desired_width = original_height, original_width

        # Resize the image
        image = cv2.resize(image, (desired_width, desired_height))

        # Display the resized image
        cv2.imshow('UltiThrow', image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the video capture object and close the window
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
# Extract landmarks

In [6]:
# Open the video file
video_path = r'videos\5_backhand_short_front_2.mp4'
cap = cv2.VideoCapture(video_path)

# Get the original width and height of the video
original_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
original_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Set up mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.6, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            print("End of video.")
            break  # Exit the loop when the video is finished.
        

        # Detect stuff and render
        
        # Recolor image
        # default of opencv is rgb, that's why we recolor
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # set the image to not be modified and save memory
        image.flags.writeable = False
        
        # Make detection
        results = pose.process(image) # get detection of pose
        
        # Recolor back to BGR for opencv
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
        except:
            pass
        
        # Render detections
        # draw_landamarks -> utils to draw in image
        # results.pose_landmarks -> coordenates
        # mp_pose.POSE_CONNECTIONS -> how body points are conencted inside model
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        # Update desired height and width before resizing
        desired_height, desired_width = original_height // 2, original_width // 2

        # Resize the image
        image = cv2.resize(image, (desired_width, desired_height))

        # Display the resized image
        cv2.imshow('UltiThrow', image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the video capture object and close the window
    cap.release()
    cv2.destroyAllWindows()

End of video.


In [14]:
len(landmarks)

33

In [21]:
for lndmrk in mp_pose.PoseLandmark:
    print(lndmrk)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32


In [26]:
landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value] # how to access the values

x: 0.50659496
y: 0.36798444
z: -0.059700783
visibility: 0.96365255

Extracting information about backhand from a rigth-hand person video taken from front perspective.

12 - right shoulder
14 - right elbow
16 - right wrist

In [27]:
landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]

x: 0.4252391
y: 0.26356584
z: -0.052073795
visibility: 0.99997765

In [28]:
landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]

x: 0.40820667
y: 0.3729021
z: -0.01889005
visibility: 0.98281825

In [29]:
landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]

x: 0.38858938
y: 0.4651661
z: -0.08842416
visibility: 0.9647229

# Calculate angles

In [7]:
def calculate_angle(a, b, c): #first, mid and end points
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1]-b[1], c[0] - b[0]) - np.arctan2(a[1]-b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
    return angle

In [8]:
shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

In [37]:
shoulder, elbow, wrist

([0.42523908615112305, 0.2635658383369446],
 [0.42523908615112305, 0.2635658383369446],
 [0.42523908615112305, 0.2635658383369446])

In [39]:
calculate_angle(shoulder, elbow, wrist)

0.0

In [26]:
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    # Vectors in 3D space
    vector1 = a - b
    vector2 = c - b
    
    dot_product = np.dot(vector1, vector2)
    norm_vector1 = np.linalg.norm(vector1)
    norm_vector2 = np.linalg.norm(vector2)
    
    # Cosine of the angle
    cos_theta = dot_product / (norm_vector1 * norm_vector2)
    
    # Ensure cos_theta is within [-1, 1] to avoid numerical errors
    cos_theta = np.clip(cos_theta, -1.0, 1.0)
    
    # Angle in radians
    radians = np.arccos(cos_theta)
    
    # Convert radians to degrees
    angle = np.degrees(radians)
    
    return angle

# Working angle for 2D

In [15]:
# Open the video file
video_path = r'videos\3_backhand_long_front.mp4'
cap = cv2.VideoCapture(0)

# Get the original width and height of the video
original_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
original_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Update desired height and width before resizing
desired_height, desired_width = original_height , original_width
# Set up mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.6, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            print("End of video.")
            break  # Exit the loop when the video is finished.
        

        # Detect stuff and render

        # Resize the image
        frame = cv2.resize(frame, (desired_width, desired_height))
        
        # Recolor image
        # default of opencv is rgb, that's why we recolor
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # set the image to not be modified and save memory
        image.flags.writeable = False
        
        # Make detection
        results = pose.process(image) # get detection of pose
        
        # Recolor back to BGR for opencv
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            shoulder_right = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            elbow_right = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            wrist_right = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            shoulder_left = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow_left = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist_left = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            # calculate angle
            angle_right = calculate_angle(shoulder_right, elbow_right, wrist_right)
            angle_left = calculate_angle(shoulder_left, elbow_left, wrist_left)
            
            #angle_right_3d = calculate_angle_3D(shoulder_right, elbow_right, wrist_right)
            #angle_left_3d = calculate_angle_3D(shoulder_left, elbow_left, wrist_left)

            # display value
            #cv2.putText(image, str(angle_right_3d), [400, 100], cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA)
            # display value
            cv2.putText(image, str(round(angle_right, 2)), 
                        tuple(np.multiply(elbow_right, [desired_width, desired_height]).astype(int)),
                              cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA)
            
            #mp_drawing.plot_landmarks(results.pose_world_landmarks, mp_pose.POSE_CONNECTIONS)
            # elbow, [original_width, original_height] -> normalized coordinates
        except Exception as e:
            print(f"Error: {e}")
            pass
        
        # Render detections
        # draw_landamarks -> utils to draw in image
        # results.pose_landmarks -> coordenates
        # mp_pose.POSE_CONNECTIONS -> how body points are conencted inside model
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        if angle_right < 90:
            cv2.putText(image, "Estira el codo", tuple(np.multiply(elbow_right, [100, 200]).astype(int)),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA)


        # Display the resized image
        cv2.imshow('UltiThrow', image)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the video capture object and close the window
    cap.release()
    cv2.destroyAllWindows()

Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'lan

Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'lan

# Cleaning data

In [10]:
# Open the video file

#filename = '4_backhand_short_side'
#video_path = f'videos//{filename}.mp4'
#csv_filename = f'{filename}_data.csv'
cap = cv2.VideoCapture(0)

# curl counter variables
counter = 0
stage = None

# Get the original width and height of the video
original_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
original_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# csv file
# csvfile.open(csv_filename, mode='w', newline='')


# Pause state variable
paused = False

# Update desired height and width before resizing
desired_height, desired_width = original_height // 2, original_width // 2
#desired_height, desired_width = original_height, original_width
# Set up mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            print("End of video.")
            break  # Exit the loop when the video is finished.
        

        # Detect stuff and render

        # Resize the image
        frame = cv2.resize(frame, (desired_width, desired_height))
        
        # Recolor image
        # default of opencv is rgb, that's why we recolor
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # set the image to not be modified and save memory
        image.flags.writeable = False
        
        # Make detection
        results = pose.process(image) # get detection of pose
        
        # Recolor back to BGR for opencv
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            shoulder_right = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            elbow_right = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            wrist_right = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            shoulder_left = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow_left = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist_left = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            
            
            
            # calculate angle
            angle_right = calculate_angle(shoulder_right, elbow_right, wrist_right)
            angle_left = calculate_angle(shoulder_left, elbow_left, wrist_left)

            # display value
            cv2.putText(image, str(f"2D: {angle_right}"), [200, 200], cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA)
            #cv2.putText(image, str(round(angle_right, 2)), 
            #            tuple(np.multiply(elbow_right, [desired_width, desired_height]).astype(int)),
            #                  cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,255,255), 2, cv2.LINE_AA)
            
            #mp_drawing.plot_landmarks(results.pose_world_landmarks, mp_pose.POSE_CONNECTIONS)
            # elbow, [original_width, original_height] -> normalized coordinates
            
        except Exception as e:
            print(f"Error: {e}")
            pass
        
        # Render detections
        # draw_landamarks -> utils to draw in image
        # results.pose_landmarks -> coordenates
        # mp_pose.POSE_CONNECTIONS -> how body points are conencted inside model
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        

        # Display the resized image
        cv2.imshow('UltiThrow', image)
        # Check for key events
        key = cv2.waitKey(1) & 0xFF
        
        if key == ord('q'):
            break  # Exit the loop if 'q' is pressed

        elif key == ord(' '):
            # Toggle the pause state when the spacebar is pressed
            paused = not paused
            mp_drawing.plot_landmarks(results.pose_world_landmarks, mp_pose.POSE_CONNECTIONS)

        # Pause the video if the paused variable is True
        if paused:
            cv2.waitKey(0)  # Wait until a key is pressed to resume

    # Release the video capture object and close the window
    cap.release()
    cv2.destroyAllWindows()

Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'lan

# Backhand-short release point

In [9]:
EXPORT_PATH = r'data\backhand_short_release_point.csv'

In [10]:
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt
import csv
import numpy as np

In [11]:
landmarks = ['class']
for val in range(1, 33+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [12]:
with open(EXPORT_PATH, mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [16]:


def export_landmark(results):
    try:
        keypoints = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten()
        keypoints = np.insert(keypoints, 0, '3')

        with open(EXPORT_PATH, mode='a', newline='') as f:
            csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)

    except Exception as e:
        print(f"An error occurred: {e}")

In [5]:
video_path = r'videos\5_backhand_short_front_2.mp4'
cap = cv2.VideoCapture(video_path)

# Get the original video width and height
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
target_height = 720

# Calculate the aspect ratio
aspect_ratio = width / height

# Calculate the new width based on the target height and original aspect ratio
target_width = int(target_height * aspect_ratio)


with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    # Variables for frame navigation
    current_frame = 0
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            print("End of video.")
            break  # Exit the loop when the video is finished.
        

        frame = cv2.resize(frame, (target_width, target_height))
        # Recolor image
        # default of opencv is rgb, that's why we recolor
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        # set the image to not be modified and save memory
        image.flags.writeable = False
        
        # Make detection
        results = pose.process(image) # get detection of pose
        
        # Recolor back to BGR for opencv
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        #landmarks = results.pose_landmarks.landmark

        cv2.imshow('Video', image)

        k = cv2.waitKey(0)  # Wait for a key press
        if k == ord('q'):
            break
        elif k == ord('n'):  # Move to the next frame
            current_frame += 1
            if current_frame >= total_frames:
                current_frame = total_frames - 1
            cap.set(cv2.CAP_PROP_POS_FRAMES, current_frame)
        elif k == ord('p'):  # Move to the previous frame
            current_frame -= 1
            if current_frame < 0:
                current_frame = 0
            cap.set(cv2.CAP_PROP_POS_FRAMES, current_frame)
        elif k == ord('r'):  # Capture the release point
            export_landmark(results)

            

    # Release the video capture object and close the window
    cap.release()
    cv2.destroyAllWindows()

Reflex exercise

In [3]:
import random 
import math

In [11]:
disk_x = int(round(random.random() * image.shape[1]))  # Random x coordinate within the image width
disk_y = int(round(random.random() * image.shape[0]))  # Random y coordinate within the image height

disk_caught = False

# VIDEO FEED

# Open the video file
#video_path = r'videos\1_backhand.mp4'
#cap = cv2.VideoCapture(video_path)
cap = cv2.VideoCapture(0)

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            # Read a frame from the video
            # ret -> return variable
            # frame -> image from cap
            ret, frame = cap.read()

            if not ret:
                print("End of video.")
                break
                
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            # set the image to not be modified and save memory
            image.flags.writeable = False
            
            image = cv2.flip(image, 1)
            
            # Make detection
            results = pose.process(image) # get detection of pose
            
            # Recolor back to BGR for opencv
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            
            try:
                landmarks = results.pose_landmarks.landmark
            except Exception as e:
                print(f"Error: {e}")
                pass
    
            right_wrist_landmarks = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            left_wrist_landmarks = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            image = cv2.circle(image, (disk_x, disk_y), radius=10, color=(0, 0, 255), thickness=-1)
            
            
            cv2.imshow('Video', image)
            
            
            # check upper hand is right
            r_wrist_x = right_wrist_landmarks[0] * image.shape[1]
            l_wrist_x = left_wrist_landmarks[0] * image.shape[1]
            
            r_wrist_y = right_wrist_landmarks[1] * image.shape[0]
            l_wrist_y = left_wrist_landmarks[1] * image.shape[0]
            
            condition_y_axis_right = r_wrist_y > l_wrist_y
            condition_visibility_points = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].visibility > 0.4 and landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].visibility > 0.4
            condition_distance = ((r_wrist_x - l_wrist_x) < 20) and ((r_wrist_x - l_wrist_x) > -20)
            
            #distance to the disk, taking as reference right wrist landmark
            distance_disk_wrist = math.sqrt((r_wrist_x - disk_x)**2 + (r_wrist_y - disk_y)**2)
            condition_disk = distance_disk_wrist < 30
            
            
            if condition_y_axis_right and condition_visibility_points and condition_distance and condition_disk:
                disk_x = int(round(random.random() * image.shape[1]))
                disk_y = int(round(random.random() * image.shape[0]))
                
            
            # Break the loop if 'q' is pressed
            k = cv2.waitKey(10)  # Wait for a key press
            if k == ord('q'):
                break
            elif k == ord('r'):  # Move to the next frame
                disk_x = int(round(random.random() * image.shape[1]))
                disk_y = int(round(random.random() * image.shape[0]))

# Release the video capture object and close the window
cap.release()
cv2.destroyAllWindows()

Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'lan

Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'lan

Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'lan

Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'lan

Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'lan

...

Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'lan

Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'lan

Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'lan

Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'lan

Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'landmark'
Error: 'NoneType' object has no attribute 'lan